In [ ]:
%load_ext autoreload
%autoreload 2

%load_ext dotenv
%dotenv

In [ ]:
import json
import random
import string

from uuid import uuid4
from pprint import pprint
from datetime import datetime, timedelta, UTC

from liqpy.client import Client
from liqpy.testing import TestCard
from liqpy.exceptions import LiqPayException

In [ ]:
client = Client()
client

In [ ]:
data, signature = client.encode(
    action="pay",
    amount=1,
    order_id=str(uuid4()),
    description="Test Encoding",
    currency="USD",
)

sep, end = "\n", "\n\n"
print("data:", data, sep=sep, end=end)
print("signature:", signature, sep=sep, end=end)

client.callback(data, signature)

In [ ]:
card_exp = datetime.now() + timedelta(days=365 * 4)

order_id = str(uuid4())
client.request(
    "pay",
    order_id=order_id,
    amount=1,
    currency="USD",
    description="Test Payment",
    card=TestCard.successful(),
    card_exp_month=str(card_exp.month).ljust(2, "0"),
    card_exp_year=str(card_exp.year)[-2:],
    card_cvv="111",
)

In [ ]:
try:
    card_exp = datetime.now() + timedelta(days=365 * 4)

    order_id = str(uuid4())
    client.request(
        "pay",
        order_id=order_id,
        amount=1,
        currency="USD",
        description="Test Payment",
        card=TestCard.failure("9859"),
        card_exp_month=str(card_exp.month).ljust(2, "0"),
        card_exp_year=str(card_exp.year)[-2:],
        card_cvv="111",
    )
except LiqPayException as e:
    print(e.response)
    pprint(e.details)
    raise e

In [ ]:
client.status(order_id)

In [ ]:
client.data(order_id, "Test info")

In [ ]:
client.checkout(
    action="pay",
    amount=1,
    order_id=str(uuid4()),
    description="Test Checkout",
    currency="USD",
)

In [ ]:
date_to = datetime.now(UTC)
date_from = date_to - timedelta(days=30)

data = client.reports(date_from=date_from, date_to=date_to, format="json")
data = json.loads(data)